# 6. 영화 리뷰 텍스트 감성 분석하기
네이버 영화 리뷰 감성 분석 도전하기   
데이터셋 [Naver sentiment movie corpus](https://github.com/e9t/nsmc)   

## 데이터 가져오기

In [4]:
import pandas as pd
 
train_data = pd.read_table('~/aiffel/sentiment_classification/data/ratings_train.txt')
test_data = pd.read_table('~/aiffel/sentiment_classification/data/ratings_test.txt')

print("train_data: {}, test_data: {}".format(len(train_data), len(test_data)))
train_data.head()

train_data: 150000, test_data: 50000


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


## 데이터 전처리

- 데이터 중복 제거
- NaN 결측치 제거
- 한국어 토큰화
- stopwords 제거
- 사전 `word_to_index` 구성
- 텍스트 스트링을 사전 인덱스 스트링으로 변환
- `X_train, y_train, X_test, y_test, word_to_index` 리턴

In [ ]:
from konlpy.tag import Mecab
import numpy as np
from collections import Counter

tokenizer = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def load_data(train_data, test_data, num_words=num_words):
    # [[YOUR CODE]]
    
    
X_train, y_train, X_test, y_test, word_to_index = load_data(train_data, test_data)

In [ ]:
# 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트 벡터로 변환해 주는 함수입니다. 
# 단, 모든 문장은 <BOS>로 시작하는 것으로 합니다. 
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index['<BOS>']]+[word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in sentence.split()]

# 여러 개의 문장 리스트를 한꺼번에 단어 인덱스 리스트 벡터로 encode해 주는 함수입니다. 
def get_encoded_sentences(sentences, word_to_index):
    return [get_encoded_sentence(sentence, word_to_index) for sentence in sentences]

# 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다. 
def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<UNK>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

# 여러 개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다. 
def get_decoded_sentences(encoded_sentences, index_to_word):
    return [get_decoded_sentence(encoded_sentence, index_to_word) for encoded_sentence in encoded_sentences]

## 데이터 분석 및 가공
- 데이터셋 내 문장 길이 분포 확인
- 적절한 최대 문장 길이 지정
- `keras.preprocessing.sequence.pad_sequences` 활용해서 패딩 추가

## 모델과 validation set 구성
__3가지 이상 다양하게 구성해보기__

## 모델 훈련

## Loss, Accuracy 그래프 시각화

## Embedding 레이어 분석

## 한국어 Word2Vec 임베딩 활용해서 성능 개선
[Pre-trained word vectors of 30+ languages](https://github.com/Kyubyong/wordvectors)   
`ko.bin` 이용하기, `gensim` 버전을 3.x.x으로 낮춰야 오류 발생하지 않음

# 회고